In [1]:
import agentpy as ap
import matplotlib.pyplot as plt
import seaborn as sns
import IPython
import random
import json
from time import time

In [2]:
parameters = {
    'm': 20, # Altura del grid
    'n': 20, # Longitud del grid
    'k': 5, # Número de carros
    'tMax': 100, # Tiempo máximo de ejecucción
    'getJson': True
}

In [3]:
class IntersectionModel(ap.Model):
    
    def setup(self):
        
        global street_v1, street_v2, street_v3, street_v4, street_v5, street_v6, streetV
        global street_h1, street_h2, street_h3, street_h4, street_h5, street_h6, streetH
        
        street_v1 = [(n, int(self.p.n/2) - 1) for n in range(0, int(self.p.m/2), 1)]
        street_v2 = [(n, int(self.p.n/2)) for n in range(0, int(self.p.m/2), 1)]
        street_v3 = [(n, int(self.p.n/2) - 1) for n in range(int(self.p.m/2), self.p.m , 1)]
        street_v4 = [(n, int(self.p.n/2)) for n in range(int(self.p.m/2), self.p.m , 1)]
        
        street_h1 = [(int(self.p.m/2) - 1, n) for n in range(0, int(self.p.n/2), 1)]
        street_h2 = [(int(self.p.m/2), n) for n in range(0, int(self.p.n/2), 1)]
        street_h3 = [(int(self.p.m/2) - 1, n) for n in range(int(self.p.n/2), self.p.n, 1)]
        street_h4 = [(int(self.p.m/2), n) for n in range(int(self.p.n/2 ), self.p.n, 1)]
        
        # Calle horizontal
        streetH = street_h1 + street_h2 + street_h3 + street_h4
        
        # Calle vertical
        streetV = street_v1 + street_v2 + street_v3 + street_v4
        
        # Generar posiciones de inicio aleatorias
        random.seed(time())
        self.pos =[(1, round(self.p.n/2) - 1), (self.p.m - 2, round(self.p.n/2)), (round(self.p.m/2), 1), (round(self.p.m/2)-1, self.p.n - 2)]
        for i in range(0,self.p.k):
            self.pos.append(self.pos[random.randint(0,3)])
       
        # Crear agentes
        self.cars = self.agents = ap.AgentList(self, self.p.k)
        
        # Crear grid
        self.street = ap.Grid(self, (self.p.m, self.p.n), track_empty = True)
        
        positions = self.pos[4:14]
        
        # Agregar agentes
        self.street.add_agents(self.cars, positions, empty = True)
        
        self.path = ap.AgentList(self,len(self.cars))
        
        # Condiciones
        # Carros: 0
        # Camino: 1
        self.agents.condition = 0
        self.path.condition = 1
        
        self.agents.orientation = 'x'
        
        # Creación del JSON
        if self.p.getJson == True:
            self.steps = 0
            self.data = {}
            self.data['steps'] = []
    
    def step(self):
        
        for i in self.cars:
            
            if(self.street.positions[i] == self.pos[0]):
                i.orientation = 'n' 
            if(self.street.positions[i] == self.pos[1]):
                i.orientation = 's'
            if(self.street.positions[i] == self.pos[2]):
                i.orientation = 'w'
            if(self.street.positions[i] == self.pos[3]):
                i.orientation = 'e'
            
            if(self.street.positions[i] == (self.p.m-2, round(self.p.n/2)-1)
              or self.street.positions[i] == (1, round(self.p.n/2))
              or self.street.positions[i] == (round(self.p.m/2), self.p.n-2)
              or self.street.positions[i] == (round(self.p.m/2)-1, 1)):
                i.orientation = 'x'
                self.street.move_to(i, self.pos[random.randint(0,3)])
            
            
            # Movimiento de carros
            if(i.orientation == 'n' 
                and not self.street.agents[self.street.positions[i][0]+1, self.street.positions[i][1]].condition == True
                and not self.street.agents[self.street.positions[i][0]+2, self.street.positions[i][1]].condition == True):
                    self.street.move_by(i, (1, 0))
            
            elif(i.orientation == 's'
               and not self.street.agents[self.street.positions[i][0]-1, self.street.positions[i][1]].condition == True
               and not self.street.agents[self.street.positions[i][0]-2, self.street.positions[i][1]].condition == True):
                    self.street.move_by(i, (-1, 0))
            
            elif(i.orientation == 'w'
               and not self.street.agents[self.street.positions[i][0], self.street.positions[i][1]+1].condition == True
               and not self.street.agents[self.street.positions[i][0], self.street.positions[i][1]+2].condition == True):
                    self.street.move_by(i, (0, 1))
            
            elif(i.orientation == 'e'
               and not self.street.agents[self.street.positions[i][0], self.street.positions[i][1]-1].condition == True
               and not self.street.agents[self.street.positions[i][0], self.street.positions[i][1]-2].condition == True):
                    self.street.move_by(i, (0, -1))
            
            
            # Giro de carros
            if(i.orientation == 'n'
                and not self.street.agents[self.street.positions[i][0]+1, self.street.positions[i][1]].condition == True
                and not self.street.agents[self.street.positions[i][0]+2, self.street.positions[i][1]].condition == True
                and ((self.street.positions[i] in street_h1) or (self.street.positions[i] in street_h3))):  
                    i.orientation = 'e'
                    self.street.move_by(i, (0, -1))
              
            elif(i.orientation == 's'
                and not self.street.agents[self.street.positions[i][0]+1, self.street.positions[i][1]].condition == True
                and not self.street.agents[self.street.positions[i][0]+2, self.street.positions[i][1]].condition == True
                and ((self.street.positions[i] in street_h2) or (self.street.positions[i] in street_h4))):
                    i.orientation = 'w'
                    self.street.move_by(i, (0, 1))
            
            elif(i.orientation == 'e'
               and not self.street.agents[self.street.positions[i][0], self.street.positions[i][1]-1].condition == True
               and not self.street.agents[self.street.positions[i][0], self.street.positions[i][1]-2].condition == True
               and ((self.street.positions[i] in street_v2) or (self.street.positions[i] in street_v4))):
                    i.orientation = 's'
                    self.street.move_by(i, (-1, 0))
        
        
        left_path = self.path.select(self.path.condition == 1)
        
        if self.t == self.p.tMax or len(left_path) == 0:
            self.stop()
        
        
        # Añadir JSON
        if self.p.getJson == True:
            
            cars_list = []
            
            for i in range(self.p.k):
                #i
                cars_list.append({
                    'id': i,
                    'x': self.street.positions[self.agents[i]][0],
                    'y': 0,
                    'z': self.street.positions[self.agents[i]][1],
                })

            self.data['steps'].append({
                
                'cars': cars_list
            })

            self.steps += 1
            
            
    def end(self):
        # Escribir archivo JSON
        if self.p.getJson == True:
            json_file = json.dumps(self.data)
            with open('data.json', 'w') as f:
                f.write(json_file)
        return

In [4]:
def animation_plot(model, ax):
    attr_grid = model.street.attr_grid('condition')
    color_dict = {0:'#D13F3F', None:'#FFFFFF'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"Simulación Intersección\n"
                 f"Tiempo: {model.t}")
fig, ax = plt.subplots()
model = IntersectionModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=5))

In [6]:
model = IntersectionModel(parameters)
results = model.run()

Completed: 100 steps
Run time: 0:00:00.069657
Simulation finished
